In [1]:
from utils import create_dataloaders
from evaluate import reconstruction_loss, evaluate_model
from VAD_Beta import VariationalAutoDecoder_Beta
from trainer import VADTrainer_Beta

In [2]:
train_ds, train_dl, test_ds, test_dl = create_dataloaders(data_path='', batch_size=64)


In [3]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

latent_dim = 128

model = VariationalAutoDecoder_Beta(latent_dim, device=device)

trainer = VADTrainer_Beta(model=model, dl=train_dl, latent_dim=model.latent_dim, device=device)

trainer.train(num_epochs=750)


# Evaluation on the test dataset
# latents = torch.nn.Parameter(torch.randn(len(test_dl.dataset), model.latent_dim).to(device))
# opt = torch.optim.Adam([latents], lr=3e-3)
# test_loss = evaluate_model(model=model, test_dl=test_dl, opt=opt, latents=latents, epochs=1000, device=device)
# print(f"AD has finished test evaluation with a test loss of {test_loss}.")

Using device: cuda
Epoch [1/750], Loss: 18453.4478
Epoch [2/750], Loss: 15777.2769
Epoch [3/750], Loss: 14021.3949
Epoch [4/750], Loss: 12947.0838
Epoch [5/750], Loss: 12331.4252
Epoch [6/750], Loss: 11932.3412
Epoch [7/750], Loss: 11777.7416
Epoch [8/750], Loss: 11686.2575
Epoch [9/750], Loss: 11608.1032
Epoch [10/750], Loss: 11518.0852
Epoch [11/750], Loss: 11385.1037
Epoch [12/750], Loss: 11287.1800
Epoch [13/750], Loss: 11216.9802
Epoch [14/750], Loss: 11166.3698
Epoch [15/750], Loss: 11109.4905
Epoch [16/750], Loss: 11019.8499
Epoch [17/750], Loss: 11009.3373
Epoch [18/750], Loss: 10876.3283
Epoch [19/750], Loss: 10758.8408
Epoch [20/750], Loss: 10728.2136
Epoch [21/750], Loss: 10644.9748
Epoch [22/750], Loss: 10619.3738
Epoch [23/750], Loss: 10585.4782
Epoch [24/750], Loss: 10452.2463
Epoch [25/750], Loss: 10406.7432
Epoch [26/750], Loss: 10274.8992
Epoch [27/750], Loss: 10261.2603
Epoch [28/750], Loss: 10146.1907
Epoch [29/750], Loss: 10081.7694
Epoch [30/750], Loss: 10039.4459


[18453.447814941406,
 15777.276916503906,
 14021.394897460938,
 12947.083801269531,
 12331.425231933594,
 11932.341247558594,
 11777.741577148438,
 11686.257507324219,
 11608.103210449219,
 11518.085205078125,
 11385.103698730469,
 11287.179992675781,
 11216.980163574219,
 11166.369750976562,
 11109.490478515625,
 11019.849914550781,
 11009.337341308594,
 10876.328308105469,
 10758.840759277344,
 10728.213623046875,
 10644.974792480469,
 10619.373779296875,
 10585.478210449219,
 10452.246337890625,
 10406.743225097656,
 10274.899169921875,
 10261.26025390625,
 10146.190673828125,
 10081.769409179688,
 10039.445861816406,
 10007.967590332031,
 9877.797607421875,
 9837.456359863281,
 9607.377319335938,
 9700.770812988281,
 9636.927368164062,
 9514.205963134766,
 9410.298095703125,
 9382.862121582031,
 9297.501831054688,
 9202.229919433594,
 9185.257080078125,
 9121.272338867188,
 9075.873657226562,
 9041.904724121094,
 8939.012603759766,
 8823.081665039062,
 8772.5234375,
 8708.862304687

In [5]:
# Randomly sample 5 indices from the test dataset
import random
import utils
from utils import plot_tsne

# Evaluation on the train dataset
latents = torch.randn((len(train_dl.dataset), model.latent_dim)).to(device)
mu_train = torch.randn(len(train_dl.dataset), latent_dim, device=device, requires_grad=True)
sigma_train = torch.randn(len(train_dl.dataset), latent_dim, device=device, requires_grad=True)
train_latents = torch.nn.parameter.Parameter(torch.stack([mu_train, sigma_train], dim=1)).to(device)
opt = torch.optim.Adam([train_latents], lr=1e-3)
train_loss = evaluate_model(model=model, test_dl=train_dl, opt=opt, latents=train_latents, epochs=800, device=device)
print(f"AD has finished test evaluation with a test loss of {train_loss}.")


# Evaluation on the test dataset
latents = torch.randn((len(test_dl.dataset), model.latent_dim)).to(device)
    
# mu_test = torch.randn(len(test_dl.dataset), latent_dim, device=device, requires_grad=True)
# sigma_test = torch.randn(len(test_dl.dataset), latent_dim, device=device, requires_grad=True)
mu_test = torch.randn(len(test_dl.dataset), latent_dim, device=device, requires_grad=True)
sigma_test = torch.randn(len(test_dl.dataset), latent_dim, device=device, requires_grad=True)
test_latents = torch.nn.parameter.Parameter(torch.stack([mu_test, sigma_test], dim=1)).to(device)
    
opt = torch.optim.Adam([test_latents], lr=1e-3)
test_loss = evaluate_model(model=model, test_dl=test_dl, opt=opt, latents=test_latents, epochs=800, device=device)
print(f"AD has finished test evaluation with a test loss of {test_loss}.")

result_test_latents = model.reparameterize(test_latents[:,0,:], test_latents[:,1,:])
utils.plot_tsne(test_ds, result_test_latents, "test TSNE_")

random.seed(6)
sampled_indices = random.sample(range(len(test_latents)), 5)
    
# Extract the corresponding vectors (input data) and their labels
sampled_latents = [test_latents[i] for i in sampled_indices]  # Only selecting input data, not labels
    
# Convert to a single tensor (optional)
sampled_latents_tensor = torch.stack(sampled_latents)
random_latents_tensor = torch.rand([5,128], device=device)
    
print("Sampled Vectors Shape:", sampled_latents_tensor.shape)  # Should be (5, *) depending on your data shape
print("Random Vectors Shape:", random_latents_tensor.shape)  # Should be (5, *) depending on your data shape
    
sampled_test_images = model(sampled_latents_tensor).view(-1, 1, 28, 28)
random_test_images = model.david_forward(random_latents_tensor).view(-1, 1, 28, 28)
    
print("Sampled Images Shape:", sampled_test_images.shape)  # Should be (5, *) depending on your data shape
utils.save_images(sampled_test_images, "sampled_test_images.png" )
utils.save_images(random_test_images, "random_test_images.png")

AD has finished test evaluation with a test loss of 2236.0185546875.
AD has finished test evaluation with a test loss of 2535.923568725586.
Sampled Vectors Shape: torch.Size([5, 2, 128])
Random Vectors Shape: torch.Size([5, 128])
Sampled Images Shape: torch.Size([5, 1, 28, 28])


<Figure size 800x600 with 0 Axes>

In [6]:
# #INTERPOLATION

sampled_latents = [result_test_latents[3],  result_test_latents[4]]
weights = [0, 0.25, 0.5, 0.75, 1]
interpolated_latents = [w * sampled_latents[0] + (1 - w) * sampled_latents[1] for w in weights]
interpolated_latents_tensor = torch.stack(interpolated_latents)
interpolated_images = model.decoder(interpolated_latents_tensor).view(-1, 1, 28, 28)
utils.save_images(interpolated_images, "interpolted_image_log-normal.png")